# Notebook de Jupyter para algoritmo ID3 - Nicole Góngora

## Creando las funciones para el procesamiento de ID3

### Hallando la entropía del dataset

In [1]:
def find_entropy(df):
    Class = df.keys()[-1] #cambiando el nombre de la clase de variables objetivo
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class]) #aplicacion de la formula de probabilidad de clases en dataset
        entropy += -fraction*np.log2(fraction) #haciendo la sumatoria para el valor de entropia
    return entropy

### Hallando la entropia de un atributo

In [2]:
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]   #cambiando el nombre de la clase de variables objetivo
    target_variables = df[Class].unique()  #Set de las variables objetivo
    variables = df[attribute].unique()    #Set de los valores por atributo
    entropy2 = 0
    for variable in variables:
        entropy = 0 #la segunda entropia de la fórmula para ganancia
        for target_variable in target_variables:
            #numero de nodos de un valor segun la variable objetivo
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            #numero de nodos de un valor
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            #calculo de la segunda entropia
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        #calculo de la primera entropia con el resultado de la otra
        entropy2 += -fraction2*entropy
    return abs(entropy2)

### Hallar el atributo para evaluar el dataset de ejemplos de entrenamiento

In [3]:
def find_winner(df):
    Entropy_att = []
    IG = []
    #Iterar los valores terminales
    for key in df.keys()[:-1]:
        #Hallar por cálculo de entropías (general y atributos terminales) los posibles
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    #Devolver el atributo con el máximo valor en el array de entropías IG
    return df.keys()[:-1][np.argmax(IG)] 


### Selección de subsets que contengan un valor deseado de un atributo

In [4]:
def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)

### Creación del árbol ID3 a partir de las formulas planteadas

In [5]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #cambiando el nombre de la clase de variables objetivo
    node = find_winner(df) #encontrar el nodo a partir del atributo escogido como el mejor, para la versión inicial
    attValue = np.unique(df[node]) #Obtener los valores únicos del atributo evaluador
    if tree is None:                    
        tree={}
        tree[node] = {}
        #Creando una estructura de árbol de manera recursiva a partir del nodo escogido, 
        #deteniendo el proceso si el subconjunto obtenido es nulo 
    for value in attValue:
        #Para cada valor de los atributos, obtener subsets a partir de este y el nodo en el que trabaja
        subtable = get_subtable(df,node,value)
        #
        clValue,counts = np.unique(subtable['play'],return_counts=True)                        
        if len(counts)==1:#Si el subset generado es de un sólo elemento, es una hoja
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) 
            #Si el subset generado es de varios elementos, llamar a la función recursivamente para extender el árbol y escoger
            #nuevo atributo candidato
    return tree

## Testing del código

In [6]:
import pandas as pd
#pd para leer la base de datos
import numpy as np
#numpy para procesos matemáticos
eps = np.finfo(float).eps
#uso del número épsilon para procedimientos más acertados
from numpy import log2 as log
#log2 para los logaritmos de base 2 de la entropia

df = pd.read_csv('tennis.csv')
print("\n Dataset de Juego de Tennis:\n\n",df)
#impresión de la tabla tennis.csv con sus atributos
tree= buildTree(df)
#creación del árbol a partir de la base de datos
import pprint
pprint.pprint(tree)
#pprint para la impresión del esquema de árbol ID3 obtenido


 Dataset de Juego de Tennis:

      Outlook Temperature Humidity    Wind play
0      Sunny         Hot     High    Weak   No
1      Sunny         Hot     High  Strong   No
2   Overcast         Hot     High    Weak  Yes
3       Rain        Mild     High    Weak  Yes
4       Rain        Cool   Normal    Weak  Yes
5       Rain        Cool   Normal  Strong   No
6   Overcast        Cool   Normal  Strong  Yes
7      Sunny        Mild     High    Weak   No
8      Sunny        Cool   Normal    Weak  Yes
9       Rain        Mild   Normal    Weak  Yes
10     Sunny        Mild   Normal  Strong  Yes
11  Overcast        Mild     High  Strong  Yes
12  Overcast         Hot   Normal    Weak  Yes
13      Rain        Mild     High  Strong   No
{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [7]:
test={'Outlook':'Sunny','Temperature':'Hot','Humidity':'High','Wind':'Weak'}
#uso de un dataset de ejemplos de entrenamiento para el testing de su rendimiento
def func(test, tree, default=None):
    attribute = next(iter(tree)) 
    print(attribute) 
    if test[attribute] in tree[attribute].keys():
        print(tree[attribute].keys())
        #imprimir los valores del atributo
        print(test[attribute])
        #imprimir el atributo
        result = tree[attribute][test[attribute]]
        #resultado de la vinculación de la clasificación del atributo con su valor, siendo iterable hasta llegar a un terminal
        if isinstance(result, dict):
            return func(test, result)
        else:
            return result
    else:
        return default
ans = func(test, tree)
#imprimir el terminal encontrado por el testeo
print(ans)

Outlook
dict_keys(['Overcast', 'Rain', 'Sunny'])
Sunny
Humidity
dict_keys(['High', 'Normal'])
High
No
